(example-ate)=

 Example: Estimating Average Treatment Effects
=============================

Motivation
----------

Estimating average treatment effects (ATEs) involves a subset of the tasks involved in estimating Conditional Average Treatment Effects (CATEs), so we can use methods that are designed for estimating CATEs to estimate ATEs. In this example, we simulate some data with confounding and demonstrate the `treatment_effect` method of the `DRLearner` class, which estimates the ATE, and compare it to estimates from some other popular libraries (`econML` and `doubleML`).

Example
-------

In [1]:
import pandas as pd
import numpy as np
import pyfixest as pf

# DGP with confounding

In [2]:
# generate covariate matrix with mixture of continuous and categorical variables
np.random.seed(123)


def dgp(n, k, pscore_fn, tau_fn, outcome_fn, k_cat=1):
    """DGP for a confounded treatment assignment dgp

    Args:
        n (int): sample size
        k (int): number of continuous covariates
        pscore_fn (lambda): propensity score function
        tau_fn (lambda): treatment effect function. Can be scalar for constant effect.
        outcome_fn (lambda): outcome DGP

    Returns:
        _type_: _description_
    """
    Sigma = np.random.uniform(-1, 1, (k, k))
    Sigma = Sigma @ Sigma.T
    Xnum = np.random.multivariate_normal(np.zeros(k), Sigma, n)
    # generate categorical variables
    Xcat = np.random.binomial(1, 0.5, (n, k_cat))
    X = np.c_[Xnum, Xcat]
    W = np.random.binomial(1, pscore_fn(X), n)
    Y = outcome_fn(X, W, tau_fn)
    df = pd.DataFrame(
        np.c_[Y, W, X], columns=["Y", "W"] + [f"X{i}" for i in range(k + 1)]
    )
    return df


pscore_fn = lambda x: 1 / (1 + np.exp(-x[:, 0] - x[:, 1] - x[:, 2] ** 2 + x[:, 3]))


# tau_fn = lambda x: 1 + 2 * x[:, 0] + 3 * x[:, 1] + 4 * x[:, 2] + 5 * x[:, 3]
def outcome_fn(x, w, taufn):
    return (
        taufn(x) * w
        + x[:, 0]
        + 2 * x[:, 1] ** 2
        + 3 * x[:, 3] * x[:, 1]
        + x[:, 2]
        + x[:, 3]
        + np.random.normal(0, 1, n)
    )


n, k = 10_000, 3
df = dgp(n, k, pscore_fn, tau_fn=lambda x: 1, outcome_fn=outcome_fn)
outcome_column, treatment_column = "Y", "W"
feature_columns = [f"X{i}" for i in range(k + 1)]

## Linear Regression (`pyfixest`)

In [3]:
print(
    naive_est := pf.feols(f"{outcome_column} ~ {treatment_column}", df)
    .tidy()
    .query("Coefficient == 'W'")
    .iloc[0, :2]
    .values
)

[1.62494388 0.0468862 ]


In [4]:
print(
    linreg_est := pf.feols(
        f"{outcome_column} ~ {treatment_column}+{'+'.join(feature_columns)}", df
    )
    .tidy()
    .query("Coefficient == 'W'")
    .iloc[0, :2].values
)

[1.132635   0.02973861]


Linear model is misspecified, so both the naive and conditional estimates are biased.

## `metalearners`: `DRLearner`

Point estimates and standard errors for treatment effects for the AIPW estimator can be computed by aggregating the pseudo-outcome computed by the `DRLearner` class.

In [5]:
from metalearners import DRLearner
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.dummy import DummyRegressor

In [6]:
metalearners_dr = DRLearner(
    nuisance_model_factory=LGBMRegressor,
    treatment_model_factory=DummyRegressor, # not actually used since we don't fit treatment model
    propensity_model_factory=LGBMClassifier,
    is_classification=False,
    n_variants=2,
    nuisance_model_params={"verbose": -1},
    propensity_model_params={"verbose": -1},
)

metalearners_dr.fit_all_nuisance(
    X=df[feature_columns],
    y=df[outcome_column],
    w=df[treatment_column],
)

metalearners_est = metalearners_dr.treatment_effect( # still need to pass data objects since DRLearner does not retain any data
    X=df[feature_columns],
    w=df[treatment_column],
    y=df[outcome_column],
)
metalearners_est

array([[1.01467296, 0.03699228]])

Manual computation with pseudo outcome method produces the same estimate (`treatment_effect` does a generalisation of this under the hood) yields the same estimate

In [7]:
gamma_i = metalearners_dr._pseudo_outcome(
    X=df[feature_columns],
    w=df[treatment_column],
    y=df[outcome_column],
    treatment_variant=1,
    is_oos=False,
)
gamma_i.mean(), gamma_i.std()/np.sqrt(n)
est, se = gamma_i.mean(), gamma_i.std()/np.sqrt(n)
print(f"est: {est}, se: {se}")

est: 1.0146729600419584, se: 0.036994131087587


## `doubleml`: `DoubleMLIRM`

The `doubleML` library focuses on estimating average effects and has an 'interactive regression model (IRM)' class that estimates the ATE using the same pseudo-outcome method as the `DRLearner` class.

In [8]:
%%capture
from doubleml import DoubleMLIRM, DoubleMLData
dml_data = DoubleMLData(
    df,
    x_cols=feature_columns,
    y_col=outcome_column,
    d_cols=treatment_column,
)

aipw_mod = DoubleMLIRM(
    dml_data,
    ml_g = LGBMRegressor(),
    ml_m = LGBMClassifier(),
    n_folds=5,
)

aipw_mod.fit()

In [9]:
print(doubleml_est := aipw_mod.summary.values[0, :2])

[1.01366331 0.03880938]


## `econML`: `LinearDRLearner`

In [10]:
from econml.dr import LinearDRLearner
import formulaic as fm

In [11]:
print(ff := f"{outcome_column} ~ 0 + {'+'.join(feature_columns)}")
y, X = fm.Formula(ff).get_model_matrix(df, output="numpy")
W = df[treatment_column].values[:, np.newaxis]

Y ~ 0 + X0+X1+X2+X3


In [12]:
%%capture
econml_dr = LinearDRLearner(model_regression=LGBMRegressor(), model_propensity=LGBMClassifier())
econml_dr.fit(y, T=W, W=X)

In [13]:
print(econml_est := econml_dr.intercept__inference(1).summary_frame().iloc[0, :2].values)

[1.069 0.059]


## comparison

All ml-based estimators yield comparable results.

In [14]:
pd.DataFrame(
 np.c_[
    naive_est,
    linreg_est,
    metalearners_est.flatten(),
    doubleml_est,
    econml_est,
], index = ['est', se],
columns = ['naive', 'linreg', 'metalearners', 'doubleml', 'econml']
)


,naive,linreg,metalearners,doubleml,econml
est,1.624944,1.132635,1.014673,1.013663,1.069
0.036994,0.046886,0.029739,0.036992,0.038809,0.059
